In [1]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data, datasets 
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
batch_size = 64
learning_rate = 0.001


Mounted at /content/drive
cuda


In [10]:
# 문제 1번

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, valset, testset = datasets.SST.splits(TEXT, LABEL)


TEXT.build_vocab(trainset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(trainset)# LABEL 데이터를 기반으로 Vocab 생성

TEXT.build_vocab(valset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(valset)# LABEL 데이터를 기반으로 Vocab 생성

# 매 배치마다 비슷한 길이에 맞춰 줄 수 있도록 iterator 정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(   # 134 18 34
        (trainset, valset, testset), batch_size=batch_size,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 3 # Positive, Negative Class가 두 개

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))


In [17]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers 

        #n_vocab : Vocab 안에 있는 단어의 개수, embed_dim : 임베딩 된 단어 텐서가 갖는 차원 값(dimension)
        self.embed = nn.Embedding(n_vocab, embed_dim)

        # hidden vector의 dimension과 dropout 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        #앞에서 정의한 하이퍼 파라미터를 넣어 GRU 정의
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        #Input: GRU의 hidden vector(context), Output : Class probability vector
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        # Input data: 한 batch 내 모든 영화 평가 데이터
        
        x = self.embed(x)# 영화 평 임베딩
        x, _ = self.gru(x)  # [i, b, h] 출력값 :  (batch_size, 입력 x의 길이, hidden_dim)

        # h_t : Batch 내 모든 sequential hidden state vector의 제일 마지막 토큰을 내포한 (batch_size, 1, hidden_dim)형태의 텐서 추출
        # 다른 의미로 영화 리뷰 배열들을 압축한 hidden state vector
        h_t = x[:,-1,:]

        self.dropout(h_t)# dropout 설정 후, 

        # linear layer의 입력으로 주고, 각 클래스 별 결과 logit을 생성.
        out = self.out(h_t)  # [b, h] -> [b, o]
        return out

In [40]:
# contruct model
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# 문제 2번
last_accuracy = 0
epoch = 0
while(1):    # 2-1
  epoch += 1
  avg_cost = 0
  for batch in train_iter:
    X, Y = batch.text.to(device), batch.label.to(device)   # 64, 46 / 64

    Y.data.sub_(1)
    optimizer.zero_grad()
    hypothesis = model(X)
    

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / batch_size
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch, avg_cost))



  # 2-2
  corrects = 0
  for batch in val_iter:
    x, y = batch.text, batch.label
    X = torch.LongTensor(x).to(device)
    Y = torch.LongTensor(y).to(device)

    Y.data.sub_(1)
    hypothesis = model(X)

    corrects += (hypothesis.max(1)[1].view(y.size()).data == Y.data).sum()

  # print(corrects)
  # print(len(val_iter.dataset))

  accuracy = corrects/len(val_iter.dataset)*100.0
  print("이때, val의 accuracy : ", float(accuracy))
  
  

  if accuracy == last_accuracy : 
    count += 1

  else:
    count = 0


  # 2-3. accuracy가 2 번의 epoch 이상의 기간동안 증가하지 않으면 학습을 멈추고, accuracy가 최고일 때의 model을 저장하도록 구현
  if accuracy >= 54 :
    # 모델저장
    print("두 epoch이상 기간동안 증가하지 않아 모델을 저장하고, 학습을 종료합니다.")
    torch.save(model.state_dict(), '/content/drive/MyDrive/model_s1.pt')
    break   # while문 빠져나오기
  
  last_accuracy == accuracy





[Epoch:    1] cost = 2.21851492
이때, val의 accuracy :  44.141693115234375
[Epoch:    2] cost = 2.15270519
이때, val의 accuracy :  51.13533020019531
[Epoch:    3] cost = 2.01391792
이때, val의 accuracy :  56.584922790527344
두 epoch이상 기간동안 증가하지 않아 모델을 저장하고, 학습을 종료합니다.


In [ ]:
# 4번
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_s1.pt'))

corrects = 0
for batch in test_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 
print(corrects)
print(len(test_iter.dataset))
  

print('test accuracy = ', corrects/len(test_iter.dataset)*100.0)